# Default config

## ResNet

In [20]:
from detectron2.config import get_cfg
from detectron2.modeling import build_resnet_backbone
from detectron2.layers import ShapeSpec

# loads default config
cfg = get_cfg()

resnet_backbone = build_resnet_backbone(cfg=cfg, input_shape=ShapeSpec(channels=3))

type(resnet_backbone)

detectron2.modeling.backbone.resnet.ResNet

In [22]:
import torch

dummy_in = torch.zeros((1, 3, 256, 256))
dummy_out = resnet_backbone(dummy_in)  # returns a dict[str -> torch.Tensor]

# only res4 here because specified in default config
for k, v in dummy_out.items():
    print(k, type(v), v.shape)

res4 <class 'torch.Tensor'> torch.Size([1, 1024, 16, 16])


## FPN

In [23]:
from detectron2.modeling.backbone.fpn import build_resnet_fpn_backbone

cfg = get_cfg()

# make changes to the default config
# (for some reason it must contain "res5", ie features after last stage)
cfg.MODEL.RESNETS.OUT_FEATURES = ['res3', 'res4', 'res5']
cfg.MODEL.FPN.IN_FEATURES = ['res3', 'res4', 'res5']

fpn_backbone = build_resnet_fpn_backbone(cfg=cfg, input_shape=ShapeSpec(channels=3))

type(fpn_backbone)

detectron2.modeling.backbone.fpn.FPN

In [24]:
dummy_in = torch.zeros((1, 3, 256, 256))
dummy_out = fpn_backbone(dummy_in)

for k,v in fpn_out.items():
    print(k, type(v), v.shape)

p3 <class 'torch.Tensor'> torch.Size([2, 256, 32, 32])
p4 <class 'torch.Tensor'> torch.Size([2, 256, 16, 16])
p5 <class 'torch.Tensor'> torch.Size([2, 256, 8, 8])
p6 <class 'torch.Tensor'> torch.Size([2, 256, 4, 4])


# Custom config

## Saving and loading a config

In [26]:
# first let's save default config to disk
import os
from pathlib import Path

default_config = get_cfg()

configs_dir = os.path.join(Path(os.path.abspath('')).parent, 'configs')
default_config_fname = os.path.join(configs_dir, 'detectron_default_config.yaml')

with open(default_config_fname, 'w') as f:
    f.write(default_config.dump())

In [27]:
# load back config from disk
from detectron2.config import CfgNode

# loads the config as a dict
default_config_dict = CfgNode.load_yaml_with_base(filename=default_config_fname)
type(default_config_dict)

dict

In [28]:
# convert dict to CfgNode
default_config_loaded = CfgNode(init_dict=default_config_dict)
type(default_config_loaded)

detectron2.config.config.CfgNode